In [8]:
import pandas as pd

In [13]:
filename = "JSD_KMB_2023-24.csv"

df = pd.read_csv(filename, thousands=',', dayfirst=True,
                 parse_dates=['Transaction Date'])

df['Transaction Date'] = pd.to_datetime(
    df['Transaction Date'], errors='coerce', dayfirst=True)

In [14]:
CATEGORIES = [
    {"name": "SpendzAccount", "keywords": ['SENT TO SPENDZ']},
    {'name': "AccomptantSalary", "keywords":  [
        "KCH AND", "IMPS TRANSACTION 230303 306204581173"]},
    {"name": "PO-RD", "keywords": ["PO RD"]},
    {"name": "Milk", "keywords": ["BABURAO RAMCHAN"]},
    {"name": "House Construction", "keywords": [
        "First Installement", "House", "Construction"]},
    {"name": "Advance Tax", "keywords": ["ETAX"]},
    {"name": "Amazon Shopping", "keywords": ["Tushar, amazon_sellet"]},
    {"name": "Cash Withdrawal", "keywords": ["ATW"]}
]


def categroize_transaction(description):
    description = str(description).lower()

    for category in CATEGORIES:
        if any(keyword.lower() in description for keyword in category['keywords']):
            return category['name']

    return 'Other'

In [15]:
df['Category'] = df['Description'].apply(categroize_transaction)

df['Debit'] = pd.to_numeric(df['Debit'], errors='coerce').fillna(0)
df['Credit'] = pd.to_numeric(df['Credit'], errors='coerce').fillna(0)
df['Amount'] = df['Credit'] - df['Debit']

df['YearMonth'] = df['Transaction Date'].dt.to_period('M')

monthly_stats = df.groupby(['YearMonth', 'Category'])[
    'Amount'].sum().reset_index()

In [16]:
monthly_stats

,YearMonth,Category,Amount
0,2023-01,Cash Withdrawal,-6000.0
1,2023-01,Milk,-1560.0
2,2023-01,Other,-310.0
3,2023-01,PO-RD,-2000.0
4,2023-01,SpendzAccount,-3000.0
...,...,...,...
64,2023-12,Cash Withdrawal,-11000.0
65,2023-12,Other,-77409.0
66,2023-12,PO-RD,-2000.0
67,2023-12,SpendzAccount,-8000.0


In [19]:
stats = monthly_stats.groupby(['YearMonth'])
stats